In [73]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
from imageio import imsave
from PIL import Image
from statistics import stdev 
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [1]:
#메모리확인

import psutil
import os

print("=="*20)
print("== memory usage check")

for exec_num in range(0, 2):
    # BEFORE code
    print(f"== {exec_num:2d} exec")
    # general RAM usage
    memory_usage_dict = dict(psutil.virtual_memory()._asdict())
    memory_usage_percent = memory_usage_dict['percent']
    print(f"BEFORE CODE: memory_usage_percent: {memory_usage_percent}%")
    # current process RAM usage
    pid = os.getpid()
    current_process = psutil.Process(pid)
    current_process_memory_usage_as_KB = current_process.memory_info()[0] / 2.**20
    print(f"BEFORE CODE: Current memory KB   : {current_process_memory_usage_as_KB: 9.3f} KB")

    X = [i for i in range(0, 9000000)]
    # AFTER  code
    memory_usage_dict = dict(psutil.virtual_memory()._asdict())
    memory_usage_percent = memory_usage_dict['percent']
    print(f"AFTER  CODE: memory_usage_percent: {memory_usage_percent}%")
    # current process RAM usage
    pid = os.getpid()
    current_process = psutil.Process(pid)
    current_process_memory_usage_as_KB = current_process.memory_info()[0] / 2.**20
    print(f"AFTER  CODE: Current memory KB   : {current_process_memory_usage_as_KB: 9.3f} KB")
    del X
    print("--"*30)

== memory usage check
==  0 exec
BEFORE CODE: memory_usage_percent: 62.6%
BEFORE CODE: Current memory KB   :    19.629 KB
AFTER  CODE: memory_usage_percent: 64.4%
AFTER  CODE: Current memory KB   :   368.125 KB
------------------------------------------------------------
==  1 exec
BEFORE CODE: memory_usage_percent: 62.7%
BEFORE CODE: Current memory KB   :    21.020 KB
AFTER  CODE: memory_usage_percent: 64.4%
AFTER  CODE: Current memory KB   :   368.199 KB
------------------------------------------------------------


In [2]:
import pandas as pd

data = pd.read_csv('/home/ncp/workspace/data/dataset_.csv',index_col=0)

In [12]:
data.head()

,identifier,sex,age_group,age,height,weight,report,pneumonia_type,orginal_dcm_file,orginal_png_file,crop_file,body_part_file,diagnosis
0,H05_00019_02,M,A1,0개월,NaN,0.9,NaN,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H05_00019_02.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4
1,H01_00716_01,F,A1,0개월,52.0,3.9,NaN,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H01_00716_01.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4
2,H03_00172_01,F,A1,0개월,42.0,1.8,NaN,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H03_00172_01.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4
3,H05_00204_01,M,A1,0개월,50.5,3.0,NaN,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H05_00204_01.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4
4,H04_00178_01,F,A1,0개월,30.0,0.5,Diffuse hazziness in both lungs. ETT insertion...,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H04_00178_01.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4


In [ ]:
data['identifier']

In [50]:
del_list2 = []

for i in del_list:
    del_list2.append(i.replace('.png',''))

def find_iden(x):
    if x in del_list2:
        return 'Y'
    else:
        return 'N'

data['del?'] = data['identifier'].apply(find_iden)
data2 = data[data['del?']=='N']

In [3]:
img_list = list(data['orginal_png_file'])
mask_list = list(data['body_part_file'])

In [58]:
import cv2
import numpy as np
def resize_img(img, size):
    out_img = cv2.resize(cv2.imread(img),size)
    return out_img

def resize_img_one(img, size):
    out_img = cv2.resize(cv2.imread(img,0),size)
    out_img = np.reshape(out_img, (128,128,1))
    return out_img

In [59]:
resize_img_one(mask_list[0],(128,128)).shape

(128, 128, 1)

In [61]:
IMG = np.ndarray((4000,128,128,1))
MASK = np.ndarray((4000,128,128,3))

for i in range(len(img_list)):
    IMG[i][:][:][:] = resize_img_one(img_list[i],(128,128))
    MASK[i][:][:][:] = resize_img(mask_list[i],(128,128))

In [62]:
from sklearn.model_selection import train_test_split

Xtr, Xtst, Ytr, Ytst = train_test_split(IMG, MASK, test_size=0.10)
Xtr, Xva, Ytr, Yva = train_test_split(IMG, MASK, test_size=0.10)


In [71]:
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Activation, Conv2D, Dropout,MaxPooling2D,UpSampling2D, Input, ELU, Concatenate, BatchNormalization
from tensorflow.python.keras.models import load_model

# Model code

first_input = Input(shape =(128,128,1))
bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(first_input)
conv1_1 = Conv2D(128, [3,3], strides=(1, 1), activation='elu', padding="same")(bn)
conv1_2=Conv2D(128, [3,3], strides=(1, 1), activation='elu', padding="same")(conv1_1)
dropout1 = Dropout(0.1)(conv1_2)
maxPool1 = MaxPooling2D(pool_size=2, strides=1)(dropout1)
conv2_1 = Conv2D(64, [3,3], strides=(2, 2), activation='elu', padding="same")(maxPool1)
conv2_2 = Conv2D(64, [3,3], strides=(1, 1), activation='elu', padding="same")(conv2_1)
dropout2 = Dropout(0.1)(conv2_2)
maxPool2 = MaxPooling2D(pool_size=2, strides=1)(dropout2)
conv3_1 = Conv2D(32, [3,3], strides=(2, 2), activation='elu', padding="same")(maxPool2)
conv3_2 = Conv2D(32, [3,3], strides=(1, 1), activation='elu', padding="same")(conv3_1)
dropout3 = Dropout(0.1)(conv3_2)
maxPool3 = MaxPooling2D(pool_size=2, strides=1)(dropout3)

conv4_1 = Conv2D(16, [3,3], strides=(2, 2), activation='elu', padding="same")(maxPool3)
conv4_2 = Conv2D(16, [3,3], strides=(1, 1), activation='elu', padding="same")(conv4_1)
dropout4 = Dropout(0.1)(conv4_2)

upSampling1 = UpSampling2D(size=(2, 2))(dropout4)

#concatenate
conv_int5 = Conv2D(32, [3,3], strides=(1, 1), activation='elu', padding="same")(upSampling1)
concat1 = Concatenate()([dropout3, conv_int5])

conv5_1 = Conv2D(32, [3,3], strides=(1, 1), activation='elu', padding="same")(concat1)
conv5_2 = Conv2D(32, [3,3], strides=(1, 1), activation='elu', padding="same")(conv5_1)
dropout5 = Dropout(0.1)(conv5_2)
upSampling2 = UpSampling2D(size=(2, 2))(dropout5)
#concatenate
conv_int6 = Conv2D(64, [3,3], strides=(1, 1), activation='elu', padding="same")(upSampling2)
concat2 = Concatenate()([dropout2, conv_int6])

conv6_1 = Conv2D(64, [3,3], strides=(1, 1), activation='elu', padding="same")(concat2)
conv6_2 = Conv2D(64, [3,3], strides=(1, 1), activation='elu', padding="same")(conv6_1)
dropout6 = Dropout(0.1)(conv6_2)
upSampling3 = UpSampling2D(size=(2, 2))(dropout6)

#concatenate
conv_int7 = Conv2D(128, [3,3], strides=(1, 1), activation='elu', padding="same")(upSampling3)
concat3 = Concatenate()([dropout1, conv_int7])

conv7_1 = Conv2D(128, [3,3], strides=(1, 1), activation='elu', padding="same")(concat3)
conv7_2 = Conv2D(128, [3,3], strides=(1, 1), activation='elu', padding="same")(conv7_1)

dropout7 = Dropout(0.1)(conv7_2)

#batch normalization
bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(dropout7)

# output = Conv2D(4, [1,1], strides=(1, 1), activation='softmax', padding="same")(bn)
output2 = Conv2D(3, [1,1], strides=(1, 1), activation='softmax', padding="same")(bn)

model = Model(inputs=first_input, outputs=output2)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# model.summary()

In [74]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_v1_4 (Batch (None, 128, 128, 1)  4           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 128, 128, 128 1280        batch_normalization_v1_4[0][0]   
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 128, 128, 128 147584      conv2d_36[0][0]                  
__________________________________________________________________________________________________
dropout_14

In [75]:
# import tensorflow as tf

# checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_test.h5', verbose=1, save_best_only = True)

# callbacks = [
#              tf.keras.callbacks.EarlyStopping(patience=2, monitor='accuracy'),
#              tf.keras.callbacks.TensorBoard(log_dir='logs')
# ]

results = model.fit(Xtr,Ytr, validation_split=0.1, batch_size=32, epochs=30)

Train on 3240 samples, validate on 360 samples
Epoch 1/30


FailedPreconditionError: Error while reading resource variable batch_normalization_v1_5/gamma from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/batch_normalization_v1_5/gamma/N10tensorflow3VarE does not exist.
	 [[{{node batch_normalization_v1_5/cond/ReadVariableOp}}]]
	 [[{{node loss_2/mul}}]]